In [1]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12419 sha256=d6f28c18a9847ae764a6341b78644af59f86216aaef8930d135a75fbbb51b57a
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
Successfully built efficientnet-pytorch


In [2]:
import os
import sys
import time
from glob import glob
from datetime import datetime

import cv2
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, BatchSampler

import torchvision
from torchvision import transforms, models

from efficientnet_pytorch import EfficientNet
import albumentations as A
from albumentations.pytorch.transforms import ToTensor
from fastprogress.fastprogress import master_bar, progress_bar

import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter("ignore")

In [49]:
train_dir = '../input/jpeg-melanoma-384x384/'
train_df = pd.read_csv('../input/jpeg-melanoma-384x384/train.csv')

In [50]:
def get_transforms(p=0.5):
    imagenet_stats = {'mean':[0.485, 0.456, 0.406], 'std':[0.229, 0.224, 0.225]}
    train_transforms = A.Compose([
        A.Cutout(p=p),
        A.RandomRotate90(p=p),
        A.Flip(p=p),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2,
                                       contrast_limit=0.2),
            A.HueSaturationValue(hue_shift_limit=20,
                                 sat_shift_limit=50,
                                 val_shift_limit=50
                                )
            
        ], p=p),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise()
        ], p=p),
        A.OneOf([
            A.MotionBlur(p=0.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1)
        ], p=p),
         A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=p),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=0.1),
            A.IAAPiecewiseAffine(p=0.3),
        ], p=p), 
        ToTensor(normalize=imagenet_stats)
    ])
    
    test_transforms =  A.Compose([
        ToTensor(normalize=imagenet_stats)
    ])
    
    return train_transforms, test_transforms

In [51]:
def get_train_val_split(df):
    df = df[df.tfrecord!=-1].reset_index(drop=True)
    train_tf_records = list(range(len(df.tfrecord.unique())))[:12]
    split_cond = df.tfrecord.apply(lambda x: x in train_tf_records)
    train_df = df[split_cond].reset_index()
    valid_df = df[~split_cond].reset_index()
    return train_df, valid_df

In [53]:
class MelanomaDataset(Dataset):
    def __init__(self, df, dataset_dir, transforms=None, is_test=False):
        super(MelanomaDataset, self).__init__()
        self.df = df
        self.dataset_dir = dataset_dir
        self.transforms = transforms
        self.is_test = is_test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image_path = f"{self.dataset_dir}/{self.df.iloc[index]['image_name']}.jpg"
        image = Image.open(image_path)
        if self.transforms is not None:
            image = self.transforms(**{'image':np.array(image)})["image"]
        if self.is_test:
            return image
        target = self.df.iloc[index]["target"]
        return image, torch.tensor([target], dtype=torch.float32)
    
    def get_labels(self):
        return list(self.df.target)

In [54]:
class EfficientNetEncoder(nn.Module):
    def __init__(self, depth=2, pool_type=F.adaptive_avg_pool2d):
        super(EfficientNetEncoder, self).__init__()
        self.pool_type = pool_type
        self.backbone = EfficientNet.from_pretrained(f'efficientnet-b{depth}')
        in_features = getattr(self.backbone, '_fc').in_features
        self.classifier = nn.Linear(in_features, 1)
    
    def forward(self, x):
        features = self.pool_type(self.backbone.extract_features(x), 1)
        features = features.view(x.size(0), -1)
        return self.classifier(features)

In [55]:
import catalyst
from catalyst.data.sampler import BalanceClassSampler

In [56]:
def get_device():
    return torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def get_model(depth='2',lr=1e-5,wd=0.01,freeze_backbone=False,opt_fn=torch.optim.AdamW,device=None):
    device = device if device else get_device()
    model = EfficientNetEncoder(depth=depth)
    if freeze_backbone:
        for parameter in model.backbone.parameters():
            parameter.requires_grad = False
    opt = opt_fn(model.parameters(),lr=lr,weight_decay=wd)
    model = model.to(device)
    return model, opt

def training_step(xb,yb,model,loss_fn,opt,device,scheduler):
    xb,yb = xb.to(device), yb.to(device)
    out = model(xb)
    opt.zero_grad()
    loss = loss_fn(out,yb)
    loss.backward()
    opt.step()
    scheduler.step()
    return loss.item()
    
def validation_step(xb,yb,model,loss_fn,device):
    xb,yb = xb.to(device), yb.to(device)
    out = model(xb)
    loss = loss_fn(out,yb)
    out = torch.sigmoid(out)
    return loss.item(),out

def get_data(train_df,valid_df,train_tfms,test_tfms,bs):
    train_ds = MelanomaDataset(df=train_df,dataset_dir='../input/jpeg-melanoma-384x384/train/',transforms=train_tfms)
    valid_ds = MelanomaDataset(df=valid_df,dataset_dir='../input/jpeg-melanoma-384x384/train/',transforms=test_tfms)
    train_dl = DataLoader(dataset=train_ds,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=4,
                          #sampler=BalanceClassSampler(labels=train_ds.get_labels(), 
                          #                           mode="downsampling")
                         )
    valid_dl = DataLoader(dataset=valid_ds,batch_size=bs*2,shuffle=False,num_workers=4)
    return train_dl,valid_dl

def fit(epochs,model,train_dl,valid_dl,opt,device=None,loss_fn=F.binary_cross_entropy_with_logits):
    
    device = device if device else get_device()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, len(train_dl)*epochs)
    val_rocs = [] 
    
    #Creating progress bar
    mb = master_bar(range(epochs))
    mb.write(['epoch','train_loss','valid_loss','val_roc'],table=True)

    for epoch in mb:    
        trn_loss,val_loss = 0.0,0.0
        val_preds = np.zeros((len(valid_dl.dataset),1))
        val_targs = np.zeros((len(valid_dl.dataset),1))
        
        #Training
        model.train()
        
        #For every batch 
        for xb,yb in progress_bar(train_dl,parent=mb):
            trn_loss += training_step(xb,yb,model,loss_fn,opt,device,scheduler) 
        trn_loss /= mb.child.total

        #Validation
        model.eval()
        with torch.no_grad():
            for i,(xb,yb) in enumerate(progress_bar(valid_dl,parent=mb)):
                loss,out = validation_step(xb,yb,model,loss_fn,device)
                val_loss += loss
                bs = xb.shape[0]
                val_preds[i*bs:i*bs+bs] = out.cpu().numpy()
                val_targs[i*bs:i*bs+bs] = yb.cpu().numpy()

        val_loss /= mb.child.total
        val_roc = roc_auc_score(val_targs.reshape(-1),val_preds.reshape(-1))
        val_rocs.append(val_roc)

        mb.write([epoch,f'{trn_loss:.6f}',f'{val_loss:.6f}',f'{val_roc:.6f}'],table=True)
    return model,val_rocs

In [59]:
df = pd.read_csv(f'../input/jpeg-melanoma-384x384/train.csv')
train_df,valid_df = get_train_val_split(df)
train_tfms,test_tfms = get_transforms(p=0.5)
train_dl,valid_dl = get_data(train_df,valid_df,train_tfms,test_tfms,bs=16)
model, opt = get_model(depth=5,lr=1e-4,wd=1e-2)

Loaded pretrained weights for efficientnet-b5


In [60]:
model,val_rocs = fit(15,model,train_dl,valid_dl,opt)

epoch,train_loss,valid_loss,val_roc


RuntimeError: CUDA out of memory. Tried to allocate 18.00 MiB (GPU 0; 15.90 GiB total capacity; 14.98 GiB already allocated; 11.88 MiB free; 15.15 GiB reserved in total by PyTorch)

In [61]:
torch.save(model.state_dict(),f'effb5.pth')

In [38]:
p=0.5
imagenet_stats = {'mean':[0.485, 0.456, 0.406], 'std':[0.229, 0.224, 0.225]}
test_tfms = A.Compose([
    A.RandomRotate90(p=p),
        A.Flip(p=p),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2,
                                       contrast_limit=0.2,
                                       ),
            A.HueSaturationValue(
                hue_shift_limit=20,
                sat_shift_limit=50,
                val_shift_limit=50)
        ], p=p),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(),
        ], p=p),
    ToTensor(normalize=imagenet_stats)
    ])

In [63]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

test_df = pd.read_csv('../input/jpeg-melanoma-384x384/test.csv')

model, opt = get_model(depth=5,lr=1e-4,wd=1e-2)
model.load_state_dict(torch.load(f'effb5.pth',map_location=device))

#Testing with lighter augmentation
test_ds = MelanomaDataset(df=test_df,dataset_dir='../input/jpeg-melanoma-384x384/test',transforms=test_tfms,is_test=True)
test_dl = DataLoader(dataset=test_ds,batch_size=16*2,shuffle=False,num_workers=4)

Loaded pretrained weights for efficientnet-b5


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 15.90 GiB total capacity; 15.08 GiB already allocated; 1.88 MiB free; 15.16 GiB reserved in total by PyTorch)

In [64]:
from tqdm import tqdm

In [43]:
def get_preds(model,device=None,tta=3):
    if device is None:
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    preds = np.zeros(len(test_ds))
    for tta_id in range(tta):
        test_preds = []
        with torch.no_grad():
            for xb in tqdm(test_dl):
                xb = xb.to(device)
                out = model(xb)
                out = torch.sigmoid(out)
                test_preds.extend(out.cpu().numpy())
            preds += np.array(test_preds).reshape(-1)
        print(f'TTA {tta_id}')
    preds /= tta
    return preds

#Changing tta to 25 from 10
preds = get_preds(model,tta=25)  

  0%|          | 0/344 [00:00<?, ?it/s]

TTA 0


100%|██████████| 344/344 [01:36<00:00,  3.57it/s]

TTA 1


In [65]:
subm = pd.read_csv('../input/jpeg-melanoma-384x384/sample_submission.csv')
subm.target = preds
subm.to_csv('submission.csv',index=False)